In [ ]:
import numpy as np
import cv2 as cv

cap = cv.VideoCapture(0)

ret, frame = cap.read()
bbox = cv.selectROI('select', frame, False)

x, y, w, h = bbox

roi = frame[y:y+h, x:x+w]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0., 60., 32.)),
                  np.array((180., 255., 255.)))
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

while(1):
    ret, frame = cap.read()

    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        ret, track_window = cv.meanShift(dst, bbox, term_crit)

        x, y, w, h = track_window
        img2 = cv.rectangle(frame, (x, y), (x+w, y+h), 255, 2)
        cv.imshow('gfg', img2)

        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cap.release()
cv.destroyAllWindows()

In [ ]:
import cv2
import time
import numpy as np

CAMERA_DEVICE_ID = 0
IMAGE_WIDTH = 320
IMAGE_HEIGHT = 240
MOTION_BLUR = True

cnt_frame = 0
fps = 0


def mse(image_a, image_b):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((image_a.astype("float") - image_b.astype("float")) ** 2)
    err /= float(image_a.shape[0] * image_a.shape[1])

    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err


def visualize_fps(image, fps: int):
    if len(np.shape(image)) < 3:
        text_color = (255, 255, 255)  # white
    else:
        text_color = (0, 255, 0)  # green
    row_size = 20  # pixels
    left_margin = 24  # pixels

    font_size = 1
    font_thickness = 1

    # Draw the FPS counter
    fps_text = 'FPS = {:.1f}'.format(fps)
    text_location = (left_margin, row_size)
    cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                font_size, text_color, font_thickness)

    return image


if __name__ == "__main__":
    try:
        # create video capture
        cap = cv2.VideoCapture(CAMERA_DEVICE_ID)

        # set resolution to 320x240 to reduce latency 
        cap.set(3, IMAGE_WIDTH)
        cap.set(4, IMAGE_HEIGHT)

        while True:
            # ----------------------------------------------------------------------
            # record start time
            start_time = time.time()
            # ----------------------------------------------------------------------
            # Read the frames from a camera
            _, frame_raw = cap.read()

            if MOTION_BLUR:
                # Denoise the frame
                frame = cv2.GaussianBlur(frame_raw, (3,3),0)
            else:
                frame = frame_raw

            # Convert to gray image
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Find edges
            edges = cv2.Canny(frame_gray,100,200)

            # Show the original and processed image
            cv2.imshow('gray', visualize_fps(frame_gray, fps))
            cv2.imshow('edge', visualize_fps(edges, fps))

            # Calculate MSE
            if cnt_frame > 0:
                if mse(frame_gray, frame_gray_p) > 100:
                    print('Frame{0}: Motion Detected!'.format(cnt_frame))

            # ----------------------------------------------------------------------
            # record end time
            end_time = time.time()

            # calculate FPS
            seconds = end_time - start_time
            fps = 1.0 / seconds
            print("Estimated fps:{0:0.1f}".format(fps));

            cnt_frame = cnt_frame + 1
            edges_p = edges
            frame_gray_p = frame_gray
            # ----------------------------------------------------------------------

            # if key pressed is 'Esc' then exit the loop
            if cv2.waitKey(1)== 27:
                break
    except Exception as e:
        print(e)
    finally:
        # Clean up and exit the program
        cv2.destroyAllWindows()
        cap.release()

Estimated fps:8.7
Frame1: Motion Detected!
Estimated fps:103.0
Estimated fps:27.9
Estimated fps:103.8
Estimated fps:55.2
Estimated fps:93.7
Estimated fps:64.2
Estimated fps:60.9
Estimated fps:60.3
Frame9: Motion Detected!
Estimated fps:27.4
Frame10: Motion Detected!
Estimated fps:93.8
Frame11: Motion Detected!
Estimated fps:63.8
Frame12: Motion Detected!
Estimated fps:62.0
Frame13: Motion Detected!
Estimated fps:62.0
Frame14: Motion Detected!
Estimated fps:75.2
Frame15: Motion Detected!
Estimated fps:59.4
Frame16: Motion Detected!
Estimated fps:65.7
Frame17: Motion Detected!
Estimated fps:66.0
Frame18: Motion Detected!
Estimated fps:56.6
Frame19: Motion Detected!
Estimated fps:91.0
Frame20: Motion Detected!
Estimated fps:58.1
Frame21: Motion Detected!
Estimated fps:73.4
Frame22: Motion Detected!
Estimated fps:68.8
Frame23: Motion Detected!
Estimated fps:61.4
Frame24: Motion Detected!
Estimated fps:72.2
Frame25: Motion Detected!
Estimated fps:66.8
Frame26: Motion Detected!
Estimated fps